In [36]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from time import sleep
import json
import pprint
from IPython.display import clear_output


In [37]:
url = "https://www.nieruchomosci-online.pl/szukaj.html?3,mieszkanie,sprzedaz"
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

In [12]:
driver.get(url)

In [4]:
# accept cookies
driver.find_element("xpath", "//span[@class='box--ci__btn box--ci__btn-highlight']").click()

In [6]:
# find first posting
driver.find_element("id", "tile_1").click()
driver.refresh()

In [40]:
#driver.find_element("id", "toplayer-nav-ul").click()

driver.find_element("css selector", "li[data-value='next']").click()

In [31]:
price = driver.find_element("xpath", "//span[@class='info-primary-price header-lg']")
float(price.text[:-3].replace(" ","").replace(",","."))

600000.0

In [29]:
m2 = driver.find_element("xpath", "//span[@class='info-area desktop-tablet-only header-lg']")
float(m2.text[:-3].replace(",","."))

22.65

In [13]:
priceperm = driver.find_element("xpath", "//span[@class='info-secondary-price desktop-tablet-only header-xs']")
float(priceperm.text[:-6].replace(" ","").replace(",","."))

10466.96

In [171]:
time = driver.find_element("css selector", "p[class='current-as desktop-tablet-only body-sm']")
time.text[16:]

'11 maja 2025'

In [ ]:
attributesTable = driver.find_element("id", "attributesTable")
attributesDivs = attributesTable.find_elements("tag name", "div")
attributes = {}
for div in attributesDivs:
    list = div.find_elements("tag name", "span")
    attribute = list[0].text
    content = list[1].text

    if(attribute == "Termin realizacji:"): # skip not yet built
        break

    attributes[attribute] = content
print(attributes)

{'Piętro:': '3', 'Liczba pokoi:': '2', 'Rok budowy:': '1984', 'Miejsce parkingowe:': '-', 'Stan mieszkania:': 'do odświeżenia'}


In [ ]:
locationTable = driver.find_element("id", "locationTable")
locationInfo = locationTable.find_elements("tag name", "span")[1:]

address = locationInfo[0].text
if len(locationInfo) < 2:
    
    #skip entry
    
surroundings = locationInfo[1].text
nearby = locationInfo[3].text

print(address)
print(surroundings)
print(nearby)

Kościuszki, Centrum, Żyrardów, żyrardowski, mazowieckie
osiedle bloków mieszkalnych, tereny zielone i rekreacyjne, obiekty handlowo-usługowe, las, rzeka
przedszkole, szkoła podstawowa, szkoła średnia, kawiarnia, boisko, apteka, przychodnia, szpital, przystanek autobusowy, dworzec autobusowy, dworzec kolejowy, supermarket, centrum handlowe, posterunek policji, bankomat, stacja benzynowa, kościół, parking, poczta, bank, park, plac zabaw


In [ ]:

dataset = []
posting_info = {}
skip = False
failures = 0
while failures < 1000:
    try:
        driver.find_element("css selector", "li[data-value='next']").click()
   
        sleep(1)
        #Price and m2
        m2 = driver.find_element("xpath", "//span[@class='info-area desktop-tablet-only header-lg']")
        price = driver.find_element("xpath", "//span[@class='info-primary-price header-lg']")

        #Attributes
        state = driver.find_element("id", "attributesTable")
        things = state.find_elements("tag name", "div")
        attributes = {}
        for thing in things:
            list = thing.find_elements("tag name", "span")
            attribute = list[0].text
            content = list[1].text

            if(attribute == "Termin realizacji:" or attribute == "Stan inwestycji:"): # skip not yet built
                skip = True
                break

            attributes[attribute] = content

        #Location info
        locationTable = driver.find_element("id", "locationTable")
        locationInfo = locationTable.find_elements("tag name", "span")[1:]

        address = locationInfo[0].text
        if len(locationInfo) < 4:
            skip = True
        else:
            surroundings = locationInfo[1].text
            nearby = len(locationInfo[3].text.split(", "))


        if skip:
            skip = False
            continue
        else:
            posting_info={
            "m2": float(m2.text[:-3].replace(",",".")),
            "price": float(price.text[:-3].replace(" ","").replace(",",".")),
            "floor": attributes.get("Piętro:", ""),
            "room_count": int(attributes.get("Liczba pokoi:", "")),
            "year_built": int(attributes.get("Rok budowy:", "")),
            "parking": attributes.get("Miejsce parkingowe:", ""),
            "state": attributes.get("Stan mieszkania:", ""),
            "address": address,
            "surroundings": surroundings,
            "nearby_count": nearby
            }
            dataset.append(posting_info)

    except Exception as e:
        failures += 1
        print(e)
    
    #clear_output(wait=True)
    print(f"Dataset size: {len(dataset)}")
    print(f"Failures: {failures}")

pprint.pprint(dataset)
with open("dataset.json", "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

Message: no such element: Unable to locate element: {"method":"css selector","selector":"li[class='offer-next-prev-nav-a']"}
  (Session info: chrome=141.0.7390.125); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
	GetHandleVerifier [0x0x7ff656c9e8e5+80021]
	GetHandleVerifier [0x0x7ff656c9e940+80112]
	(No symbol) [0x0x7ff656a2060f]
	(No symbol) [0x0x7ff656a78854]
	(No symbol) [0x0x7ff656a78b1c]
	(No symbol) [0x0x7ff656acc927]
	(No symbol) [0x0x7ff656aa126f]
	(No symbol) [0x0x7ff656ac968a]
	(No symbol) [0x0x7ff656aa1003]
	(No symbol) [0x0x7ff656a695d1]
	(No symbol) [0x0x7ff656a6a3f3]
	GetHandleVerifier [0x0x7ff656f5dc7d+2960429]
	GetHandleVerifier [0x0x7ff656f57f3a+2936554]
	GetHandleVerifier [0x0x7ff656f78977+3070247]
	GetHandleVerifier [0x0x7ff656cb83ce+185214]
	GetHandleVerifier [0x0x7ff656cbfe1f+216527]
	GetHandleVerifier [0x0x7ff656ca7b24+117460]
	GetHandleVerifier [0x0x7ff656c

In [35]:
import pprint
import json
pprint.pprint(dataset)
with open("dataset.json", "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)


[{'address': 'Flisaków, Kobiałka, Warszawa, mazowieckie',
  'floor': 'parter',
  'm2': 65.62,
  'nearby_count': 27,
  'parking': 'w garażu podziemnym',
  'price': 895000.0,
  'room_count': 3,
  'state': 'wysoki standard',
  'surroundings': 'osiedle bloków mieszkalnych, łąka, las, rzeka',
  'year_built': 2019},
 {'address': 'Geodetów 6, Ochota, Warszawa, mazowieckie',
  'floor': '1',
  'm2': 24.0,
  'nearby_count': 33,
  'parking': 'parking publiczny / na ulicy',
  'price': 540000.0,
  'room_count': 1,
  'state': 'bardzo dobry',
  'surroundings': 'osiedle bloków mieszkalnych, tereny zielone i rekreacyjne, '
                  'obiekty handlowo-usługowe',
  'year_built': 1967},
 {'address': 'Walońska, Przedmieście Świdnickie, Wrocław, dolnośląskie',
  'floor': '4',
  'm2': 58.45,
  'nearby_count': 26,
  'parking': 'w garażu podziemnym',
  'price': 1249000.0,
  'room_count': 2,
  'state': 'wysoki standard',
  'surroundings': 'osiedle bloków mieszkalnych, tereny zielone i rekreacyjne, '
   